![image](https://github.com/ecastillot/SeisMonitor/blob/master/docs/figures/seismonitor.PNG?raw=true)

**This code is necessary on colab to install SeisMonitor.** 

In [ ]:
import sys
if 'google.colab' in sys.modules:
    !pip install SeisMonitor
    !pip install git+https://github.com/ecastillot/EQTransformer.git@master
    !pip install git+https://github.com/wayneweiqiang/GaMMA.git

**Please restart the kernel. It's mandatory to get everything up and running.**

Take a look at the example data 

In [1]:
import os
import zipfile
from SeisMonitor.utils4examples import clone_seismonitor_data

dataset = "../seismonitor_dataset"

clone_seismonitor_data(dataset,branch="dataset")
with zipfile.ZipFile(os.path.join(dataset,"downloads.zip"), 'r') as zip_ref:
    zip_ref.extractall(dataset)

downloads = os.path.join(dataset,"downloads")
stations = os.path.join(dataset,"stations")

print("Dataset dir: ",dataset)
print("Important folders in your dataset",["downloads","stations"])
print("#"*100)
print("downloads")
print("\t",os.listdir(downloads))
print("stations")
print("\t",os.listdir(stations))

Dataset dir:  ../seismonitor_dataset
Important folders in your dataset ['downloads', 'stations']
####################################################################################################
downloads
	 ['PRA', 'YOT', 'ORTC', 'CHI', 'GARC', 'FLO2', 'CLEJA', 'URMC']
stations
	 ['inv.xml', 'stations.json']


# Picking with EQTransformer

In [2]:
import os
from SeisMonitor.monitor.picker.ai import EQTransformer,EQTransformerObj
from SeisMonitor.monitor.picker import utils as piut

2023-05-09 21:44:29.995621: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-09 21:44:32.404316: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-09 21:44:32.405680: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-09 21:44:35.470973: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Cloning EQTransformer repository to use the original EQTransformer pretrained model

In [4]:
piut.git_clone_aipicker(name="EQTransformer",repo_dir=OUTPUT)

eqt_path = os.path.join(OUTPUT,"EQTransformer")
if os.path.isdir(eqt_path):
    print(True)

There is already
True


We create the object: EQTransformerObj, which contains the EQTransformer parameters.

In [5]:
eqtobj = EQTransformerObj(eqt_path = eqt_path,
            n_processor = 6,
            overlap = 0.3,
            detection_threshold =0.1,
            P_threshold = 0.01,
            S_threshold = 0.01,
            batch_size = 20,
            number_of_plots = 0,
            plot_mode = 1 ) 

- EQTransformer is instanced with EQTransformerObj.
- We use **pick** method to pick seismic phases. The input parameters are:
    - Folder where is downloaded the data,
    - Folder where is downloaded the metadata 
    - Output folder from EQTransformer
- We use eqt_picks_2_seismonitor_fmt to export seismic phases in specific format util for the rest of the workflow.

In [8]:
mseed_storage = os.path.join(download_folder,"archive")
json_dir = os.path.join(download_folder,"stations")
out_dir = os.path.join(OUTPUT,"picks","eqt")
result = os.path.join(OUTPUT,"picks","eqt","seismonitor_picks.csv")

eqt = EQTransformer(eqtobj)
eqt.pick(mseed_storage,json_dir,out_dir)
piut.eqt_picks_2_seismonitor_fmt(out_dir,mseed_storage,result)

05-09 21:45 [INFO] [SeisMonitor.Picker: EQTransfomer] Running EQTransformer
05-09 21:45 [INFO] [EQTransformer] Running EqTransformer  0.1.61
05-09 21:45 [INFO] [EQTransformer] *** Loading the model ...
05-09 21:45 [DEBUG] [h5py._conv] Creating converter from 3 to 5
05-09 21:45 [WARNING] [absl] `lr` is deprecated in Keras optimizer, please use `learning_rate` or use the legacy optimizer, e.g.,tf.keras.optimizers.legacy.Adam.
05-09 21:45 [INFO] [EQTransformer] *** Loading is complete!
05-09 21:45 [INFO] [EQTransformer] *** /home/emmanuel/EDCT/SeisMonitor/examples/OUTPUT/picks/eqt/results already exists!
05-09 21:45 [INFO] [EQTransformer] There are files for 7 stations in /home/emmanuel/EDCT/SeisMonitor/examples/OUTPUT/downloads/archive directory.
05-09 21:45 [INFO] [EQTransformer] Started working on CHI, 1 out of 7 ...
05-09 21:45 [INFO] [EQTransformer] 20191224T185959Z__20191224T205959Z.mseed
2023-05-09 21:45:44.383978: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] 

aca


""


# Picking with PhaseNet

In [4]:
import os
from SeisMonitor.monitor.picker.ai import PhaseNet,PhaseNetObj
from SeisMonitor.monitor.picker import utils as piut

Using TensorFlow backend.


Cloning PhaseNet repository to use the original PhaseNet pretrained model

In [5]:
piut.git_clone_aipicker(name="PhaseNet",repo_dir=OUTPUT)

pnet_path = os.path.join(OUTPUT,"PhaseNet")
if os.path.isdir(pnet_path):
    print(True)

There is alaready
True


We create the object: PhaseNetObj, which contains the PhaseNet parameters.

In [6]:
pnetobj = PhaseNetObj(pnet_path,
            P_threshold=0.7, S_threshold=0.6,
            batch_size=100
            ) 

- PhaseNet is instanced with PhaseNetObj.
- We use **pick** method to pick seismic phases. The input parameters are:
    - Folder where is downloaded the data,
    - Folder where is downloaded the metadata 
    - Output folder from PhaseNet

In [7]:
mseed_storage = os.path.join(download_folder,"archive")
json_dir = os.path.join(download_folder,"stations")
out_dir = os.path.join(OUTPUT,"picks","pnet")
result = os.path.join(OUTPUT,"picks","pnet","seismonitor_picks.csv")

pnet = PhaseNet(pnetobj)
pnet.pick(mseed_storage,json_dir,out_dir)

01-18 16:00 [INFO] [SeisMonitor.Picker: PhaseNet] Moving mseed2onefolder in 0.258751 seconds.
01-18 16:00 [INFO] [SeisMonitor.Picker: PhaseNet] Running to create datalist
01-18 16:02 [INFO] [SeisMonitor.Picker: PhaseNet] Datalist created in 125.94694 seconds.
01-18 16:02 [INFO] [SeisMonitor.Picker: PhaseNet] Running PhaseNet pretrained model.
01-18 16:02 [INFO] [SeisMonitor.Picker: PhaseNet] Console-> python /home/emmanuel/SeisMonitor/examples/PhaseNet/run.py --mode=pred             --model_dir=/home/emmanuel/SeisMonitor/examples/PhaseNet/model/190703-214543 --data_dir=/home/emmanuel/SeisMonitor/out/picks/pnet/datadir             --data_list=/home/emmanuel/SeisMonitor/out/picks/pnet/datalist/fname.csv --output_dir=/home/emmanuel/SeisMonitor/out/picks/pnet/results            --batch_size=100 --tp_prob=0.7            --ts_prob=0.6 --resampling=-1 --input_mseed
01-18 16:02 [INFO] [SeisMonitor.Picker: PhaseNet] Running...
2023-01-18 16:02:48,021 Pred log: /home/emmanuel/SeisMonitor/out/pic

Last batch: 40 samples
Done


01-18 16:08 [INFO] [SeisMonitor.Picker: PhaseNet] Duplicated picks had been removed. See your results in /home/emmanuel/SeisMonitor/out/picks/pnet/results/seismonitor_picks.csv
01-18 16:08 [INFO] [SeisMonitor.Picker: PhaseNet] Time of prediction: 333.657172 seconds
01-18 16:08 [INFO] [SeisMonitor.Picker: PhaseNet] Moving mseed2stationfolder in 0.002016 seconds.


,pick_id,arrival_time,probability,phasehint,network,station,location,instrument_type,author,creation_time,...,station_lat,station_lon,station_elv,file_name,mseed_start_time,mseed_end_time,sampling_rate,sample,segment,segment_type
0,20191224.190140.43-S-CM.GARC.0.HH,2019-12-24 19:01:40.438394,0.749797,S,CM,GARC,0,HH,PhaseNet,2023-01-18 21:07:28.899509,...,2.186667,-75.493167,1999.0,/home/emmanuel/SeisMonitor/out/picks/pnet/data...,2019-12-24 18:59:59.998394,2019-12-24 20:59:59.998394,100.0,2544,729000,overlap
1,20191224.190236.07-P-CM.URMC.0.HH,2019-12-24 19:02:36.078394,0.768917,P,CM,URMC,0,HH,PhaseNet,2023-01-18 21:07:41.142477,...,3.247190,-74.378790,725.0,/home/emmanuel/SeisMonitor/out/picks/pnet/data...,2019-12-24 18:59:59.998394,2019-12-24 20:59:59.998394,100.0,608,15000,single
2,20191224.190236.07-P-CM.URMC.0.HH,2019-12-24 19:02:36.078394,0.735167,P,CM,URMC,0,HH,PhaseNet,2023-01-18 21:07:45.315319,...,3.247190,-74.378790,725.0,/home/emmanuel/SeisMonitor/out/picks/pnet/data...,2019-12-24 18:59:59.998394,2019-12-24 20:59:59.998394,100.0,2108,735000,overlap
3,20191224.190240.26-S-CM.URMC.0.HH,2019-12-24 19:02:40.268393,0.708441,S,CM,URMC,0,HH,PhaseNet,2023-01-18 21:07:19.519889,...,3.247190,-74.378790,725.0,/home/emmanuel/SeisMonitor/out/picks/pnet/data...,2019-12-24 18:59:59.998393,2019-12-24 20:59:59.998393,100.0,2527,735000,overlap
4,20191224.190240.28-S-CM.URMC.0.HH,2019-12-24 19:02:40.288393,0.821823,S,CM,URMC,0,HH,PhaseNet,2023-01-18 21:07:15.694105,...,3.247190,-74.378790,725.0,/home/emmanuel/SeisMonitor/out/picks/pnet/data...,2019-12-24 18:59:59.998393,2019-12-24 20:59:59.998393,100.0,1029,15000,single
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6843,20191225.005941.10-S-CM.URMC.0.HH,2019-12-25 00:59:41.108393,0.689340,S,CM,URMC,0,HH,PhaseNet,2023-01-18 21:08:07.522718,...,3.247190,-74.378790,725.0,/home/emmanuel/SeisMonitor/out/picks/pnet/data...,2019-12-24 22:59:59.998393,2019-12-25 00:59:59.998393,100.0,2611,1437000,overlap
6844,20191225.005953.10-P-CM.URMC.0.HH,2019-12-25 00:59:53.108393,0.930672,P,CM,URMC,0,HH,PhaseNet,2023-01-18 21:07:52.040484,...,3.247190,-74.378790,725.0,/home/emmanuel/SeisMonitor/out/picks/pnet/data...,2019-12-24 22:59:59.998393,2019-12-25 00:59:59.998393,100.0,2311,717000,single
6845,20191225.005953.18-P-CM.URMC.0.HH,2019-12-25 00:59:53.188393,0.955563,P,CM,URMC,0,HH,PhaseNet,2023-01-18 21:06:57.398456,...,3.247190,-74.378790,725.0,/home/emmanuel/SeisMonitor/out/picks/pnet/data...,2019-12-24 22:59:59.998393,2019-12-25 00:59:59.998393,100.0,2319,717000,single
6846,20191225.005953.21-P-CM.URMC.0.HH,2019-12-25 00:59:53.218393,0.954471,P,CM,URMC,0,HH,PhaseNet,2023-01-18 21:08:05.314880,...,3.247190,-74.378790,725.0,/home/emmanuel/SeisMonitor/out/picks/pnet/data...,2019-12-24 22:59:59.998393,2019-12-25 00:59:59.998393,100.0,2322,717000,single
